In [10]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [11]:
from datasets import load_dataset
from google.colab import drive
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity

sns.set()

/usr/local/lib/python3.10/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Use 'google/mt5-small' for non-pro cloab users
model_repo = 'google/mt5-small'
model_path = '/content/gdrive/My Drive/mt5_translation.pt'
max_seq_len = 20

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Model description: https://huggingface.co/google/mt5-base
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

In [ ]:
# max_new_tokens = 20
token_ids = tokenizer.encode(
    ' This will be translated to Japanese! (hopefully)',
    return_tensors='pt').cuda()
print(token_ids)

model_out = model.generate(token_ids)
print(model_out)

output_text = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_out[0]))
print(output_text)
     


tensor([[  1494,    898,    390,  37194,    285,    288,  30865,    309,    274,
         116024,  11994,    271,      1]], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,      1]], device='cuda:0')
<pad> <extra_id_0></s>


In [ ]:
example_input_str = ' This is just a test nbuig.'
# example_input_str = 'これは普通のテスト'
input_ids = tokenizer.encode(example_input_str, return_tensors='tf')
print('Input IDs:', input_ids)

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)

sorted(tokenizer.vocab.items(), key=lambda x: x[1])


Input IDs: tf.Tensor([[1494  339 1627  259  262 2978  259  272 1982 1315  260    1]], shape=(1, 12), dtype=int32)
Tokens: ['▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>']


[('<pad>', 0),
 ('</s>', 1),
 ('<unk>', 2),
 ('<0x00>', 3),
 ('<0x01>', 4),
 ('<0x02>', 5),
 ('<0x03>', 6),
 ('<0x04>', 7),
 ('<0x05>', 8),
 ('<0x06>', 9),
 ('<0x07>', 10),
 ('<0x08>', 11),
 ('<0x09>', 12),
 ('<0x0A>', 13),
 ('<0x0B>', 14),
 ('<0x0C>', 15),
 ('<0x0D>', 16),
 ('<0x0E>', 17),
 ('<0x0F>', 18),
 ('<0x10>', 19),
 ('<0x11>', 20),
 ('<0x12>', 21),
 ('<0x13>', 22),
 ('<0x14>', 23),
 ('<0x15>', 24),
 ('<0x16>', 25),
 ('<0x17>', 26),
 ('<0x18>', 27),
 ('<0x19>', 28),
 ('<0x1A>', 29),
 ('<0x1B>', 30),
 ('<0x1C>', 31),
 ('<0x1D>', 32),
 ('<0x1E>', 33),
 ('<0x1F>', 34),
 ('<0x20>', 35),
 ('<0x21>', 36),
 ('<0x22>', 37),
 ('<0x23>', 38),
 ('<0x24>', 39),
 ('<0x25>', 40),
 ('<0x26>', 41),
 ('<0x27>', 42),
 ('<0x28>', 43),
 ('<0x29>', 44),
 ('<0x2A>', 45),
 ('<0x2B>', 46),
 ('<0x2C>', 47),
 ('<0x2D>', 48),
 ('<0x2E>', 49),
 ('<0x2F>', 50),
 ('<0x30>', 51),
 ('<0x31>', 52),
 ('<0x32>', 53),
 ('<0x33>', 54),
 ('<0x34>', 55),
 ('<0x35>', 56),
 ('<0x36>', 57),
 ('<0x37>', 58),
 ('<0x38>',

In [ ]:
dataset = load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

Dataset alt downloaded and prepared to /root/.cache/huggingface/datasets/alt/alt-parallel/1.0.0/b498af66d7e1b78d98b7557e7b0b06ae54fc60491dccdc1a0f8316e677d1ce75. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'hi': '<hi>',
    'zh': '<zh>'
}

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))


Embedding(250103, 512)

In [ ]:
token_ids = tokenizer.encode(
    example_input_str, return_tensors='pt', padding='max_length',
    truncation=True, max_length=max_seq_len)
print(token_ids)

tokens = tokenizer.convert_ids_to_tokens(token_ids[0])
print(tokens)


tensor([[1494,  339, 1627,  259,  262, 2978,  259,  272, 1982, 1315,  260,    1,
            0,    0,    0,    0,    0,    0,    0,    0]])
['▁This', '▁is', '▁just', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      # text = '<hi>' + 'hoi supp'
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]
  
def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)
  
  return token_ids[0]

def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  # Choose a random 2 languages for in i/o
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)
  
  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)
    
    if formatted_data is None:
      continue
    
    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))
    
  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

# generates data 
def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)


In [ ]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train_dataset[0]['translation'], LANG_TOKEN_MAPPING, tokenizer)

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)


<hi> ▁ 意大利 在 法国 巴黎 王子 公园 体育 场 举办 的 2007 年 橄 榄 球 世界杯 C 组 以 3 1-5 击 败 葡萄 牙 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁2007 ▁में ▁फ़ ्रा ंस , ▁पे रिस ▁के ▁पार् क ▁डे स ▁ प्रि ंस ेस ▁में ▁हु ए ▁ र ग् बी ▁विश्व ▁कप ▁के ▁पू ल ▁C ▁में ▁ इट ली ▁ने ▁पुर् त गाल ▁को ▁3 1-5 ▁से ▁हर ाया । </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [ ]:
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
# Constants
n_epochs = 8
batch_size = 16
print_freq = 50
checkpoint_freq = 1000
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)


In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, n_warmup_steps, total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []


In [ ]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)


In [ ]:
# for epoch_idx in range(n_epochs):
#   # Randomize data order
#   data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING,
#                                       tokenizer, batch_size)
                
#   for batch_idx, (input_batch, label_batch) \
#       in tqdm_notebook(enumerate(data_generator), total=n_batches):
#     optimizer.zero_grad()

#     # Forward pass
#     model_out = model.forward(
#         input_ids = input_batch,
#         labels = label_batch)

#     # Calculate loss and update weights
#     loss = model_out.loss
#     losses.append(loss.item())
#     loss.backward()
#     optimizer.step()
#     scheduler.step()

#     # Print training update info
#     if (batch_idx + 1) % print_freq == 0:
#       avg_loss = np.mean(losses[-print_freq:])
#       print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
#           epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))
      
#     if (batch_idx + 1) % checkpoint_freq == 0:
#       test_loss = eval_model(model, test_dataset)
#       print('Saving model with test loss of {:.3f}'.format(test_loss))
#       torch.save(model.state_dict(), model_path)

# torch.save(model.state_dict(), model_path)


<ipython-input-21-7d82fc3c8b16>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 1 | Step: 50 | Avg. loss: 16.010 | lr: 0.0002777777777777778
Epoch: 1 | Step: 100 | Avg. loss: 6.916 | lr: 0.0004994418396963608
Epoch: 1 | Step: 150 | Avg. loss: 5.108 | lr: 0.0004966510381781648
Epoch: 1 | Step: 200 | Avg. loss: 4.506 | lr: 0.0004938602366599688
Epoch: 1 | Step: 250 | Avg. loss: 4.216 | lr: 0.0004910694351417727
Epoch: 1 | Step: 300 | Avg. loss: 4.107 | lr: 0.0004882786336235767
Epoch: 1 | Step: 350 | Avg. loss: 3.936 | lr: 0.0004854878321053807
Epoch: 1 | Step: 400 | Avg. loss: 3.779 | lr: 0.00048269703058718467
Epoch: 1 | Step: 450 | Avg. loss: 3.738 | lr: 0.0004799062290689886
Epoch: 1 | Step: 500 | Avg. loss: 3.654 | lr: 0.00047711542755079264
Epoch: 1 | Step: 550 | Avg. loss: 3.581 | lr: 0.00047432462603259657
Epoch: 1 | Step: 600 | Avg. loss: 3.589 | lr: 0.00047153382451440055
Epoch: 1 | Step: 650 | Avg. loss: 3.452 | lr: 0.00046874302299620453
Epoch: 1 | Step: 700 | Avg. loss: 3.440 | lr: 0.00046595222147800846
Epoch: 1 | Step: 750 | Avg. loss: 3.402 | 

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 2 | Step: 50 | Avg. loss: 3.125 | lr: 0.0004391047108729627
Epoch: 2 | Step: 100 | Avg. loss: 3.002 | lr: 0.00043631390935476675
Epoch: 2 | Step: 150 | Avg. loss: 3.107 | lr: 0.0004335231078365707
Epoch: 2 | Step: 200 | Avg. loss: 3.011 | lr: 0.0004307323063183746
Epoch: 2 | Step: 250 | Avg. loss: 3.036 | lr: 0.00042794150480017864
Epoch: 2 | Step: 300 | Avg. loss: 3.045 | lr: 0.00042515070328198257
Epoch: 2 | Step: 350 | Avg. loss: 3.063 | lr: 0.00042235990176378655
Epoch: 2 | Step: 400 | Avg. loss: 2.952 | lr: 0.00041956910024559054
Epoch: 2 | Step: 450 | Avg. loss: 3.046 | lr: 0.0004167782987273945
Epoch: 2 | Step: 500 | Avg. loss: 2.956 | lr: 0.0004139874972091985
Epoch: 2 | Step: 550 | Avg. loss: 3.018 | lr: 0.0004111966956910025
Epoch: 2 | Step: 600 | Avg. loss: 2.968 | lr: 0.0004084058941728064
Epoch: 2 | Step: 650 | Avg. loss: 2.965 | lr: 0.00040561509265461045
Epoch: 2 | Step: 700 | Avg. loss: 2.970 | lr: 0.0004028242911364144
Epoch: 2 | Step: 750 | Avg. loss: 2.950 | l

...

Epoch: 6 | Step: 50 | Avg. loss: 2.305 | lr: 0.00018659298950658628
Epoch: 6 | Step: 100 | Avg. loss: 2.352 | lr: 0.00018380218798839026
Epoch: 6 | Step: 150 | Avg. loss: 2.319 | lr: 0.00018101138647019424
Epoch: 6 | Step: 200 | Avg. loss: 2.350 | lr: 0.00017822058495199823
Epoch: 6 | Step: 250 | Avg. loss: 2.351 | lr: 0.0001754297834338022
Epoch: 6 | Step: 300 | Avg. loss: 2.319 | lr: 0.00017263898191560617
Epoch: 6 | Step: 350 | Avg. loss: 2.325 | lr: 0.00016984818039741015
Epoch: 6 | Step: 400 | Avg. loss: 2.375 | lr: 0.0001670573788792141
Epoch: 6 | Step: 450 | Avg. loss: 2.384 | lr: 0.0001642665773610181
Epoch: 6 | Step: 500 | Avg. loss: 2.296 | lr: 0.00016147577584282204
Epoch: 6 | Step: 550 | Avg. loss: 2.281 | lr: 0.00015868497432462603
Epoch: 6 | Step: 600 | Avg. loss: 2.288 | lr: 0.00015589417280643
Epoch: 6 | Step: 650 | Avg. loss: 2.296 | lr: 0.000153103371288234
Epoch: 6 | Step: 700 | Avg. loss: 2.368 | lr: 0.00015031256977003795
Epoch: 6 | Step: 750 | Avg. loss: 2.312 | l

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 7 | Step: 50 | Avg. loss: 2.202 | lr: 0.0001234650591649922
Epoch: 7 | Step: 100 | Avg. loss: 2.218 | lr: 0.00012067425764679617
Epoch: 7 | Step: 150 | Avg. loss: 2.194 | lr: 0.00011788345612860014
Epoch: 7 | Step: 200 | Avg. loss: 2.261 | lr: 0.0001150926546104041
Epoch: 7 | Step: 250 | Avg. loss: 2.193 | lr: 0.00011230185309220809
Epoch: 7 | Step: 300 | Avg. loss: 2.237 | lr: 0.00010951105157401206
Epoch: 7 | Step: 350 | Avg. loss: 2.204 | lr: 0.00010672025005581604
Epoch: 7 | Step: 400 | Avg. loss: 2.167 | lr: 0.00010392944853762
Epoch: 7 | Step: 450 | Avg. loss: 2.177 | lr: 0.00010113864701942398
Epoch: 7 | Step: 500 | Avg. loss: 2.210 | lr: 9.834784550122795e-05
Epoch: 7 | Step: 550 | Avg. loss: 2.213 | lr: 9.555704398303193e-05
Epoch: 7 | Step: 600 | Avg. loss: 2.241 | lr: 9.27662424648359e-05
Epoch: 7 | Step: 650 | Avg. loss: 2.186 | lr: 8.997544094663988e-05
Epoch: 7 | Step: 700 | Avg. loss: 2.226 | lr: 8.718463942844385e-05
Epoch: 7 | Step: 750 | Avg. loss: 2.267 | lr: 

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 8 | Step: 50 | Avg. loss: 2.157 | lr: 6.0337128823398084e-05
Epoch: 8 | Step: 100 | Avg. loss: 2.165 | lr: 5.754632730520205e-05
Epoch: 8 | Step: 150 | Avg. loss: 2.120 | lr: 5.475552578700603e-05
Epoch: 8 | Step: 200 | Avg. loss: 2.067 | lr: 5.196472426881e-05
Epoch: 8 | Step: 250 | Avg. loss: 2.085 | lr: 4.9173922750613974e-05
Epoch: 8 | Step: 300 | Avg. loss: 2.128 | lr: 4.638312123241795e-05
Epoch: 8 | Step: 350 | Avg. loss: 2.168 | lr: 4.3592319714221926e-05
Epoch: 8 | Step: 400 | Avg. loss: 2.115 | lr: 4.08015181960259e-05
Epoch: 8 | Step: 450 | Avg. loss: 2.082 | lr: 3.801071667782988e-05
Epoch: 8 | Step: 500 | Avg. loss: 2.149 | lr: 3.521991515963385e-05
Epoch: 8 | Step: 550 | Avg. loss: 2.134 | lr: 3.2429113641437824e-05
Epoch: 8 | Step: 600 | Avg. loss: 2.152 | lr: 2.9638312123241796e-05
Epoch: 8 | Step: 650 | Avg. loss: 2.112 | lr: 2.684751060504577e-05
Epoch: 8 | Step: 700 | Avg. loss: 2.126 | lr: 2.405670908684974e-05
Epoch: 8 | Step: 750 | Avg. loss: 2.126 | lr: 2.

In [ ]:
test_sentence = test_dataset[0]['translation']['en']
print('Raw input text:', test_sentence)

input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'hi',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

print('Truncated input text:', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw input text: It has been confirmed that eight thoroughbred race horses at Randwick Racecourse in Sydney have been infected with equine influenza.
Truncated input text: <hi> It has been confirmed that eight thoroughbred race horses at Randwick Racecourse</s>


In [ ]:
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)
# print(output_tokens)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

यह पुष्टि हुई है कि रविवार को रग्बी रेस के
यह पुष्टि हुई है कि रग्बी रेस के दौरान 
यह पुष्टि हुई है कि रविवार को रग्बी रेस में


In [ ]:
print(output_tokens[1])

tensor([     0,    313, 124754,   2488,  59788,  40512,   9462,   2488,  59788,
          1890,   3526,   5073,   3286,    259,   9381,    261,   7556,   9332,
           259,   7813], device='cuda:0')


In [ ]:
# initialize an empty list to store truncated input texts
truncated_inputs = []
none_indices = []
hindi_sentences = []

# iterate over each sentence in the test dataset
for i, example in enumerate(test_dataset):
    test_sentence = example['translation']['en']
    
    if test_sentence is None:
        none_indices.append(i)  # add the index to the none_indices list
        continue  # skip over this sentence and move on to the next
    
    # print('Raw input text:', test_sentence)

    # encode the input text and convert it to tensor
    input_ids1 = encode_input_str(
        text = test_sentence,
        target_lang = 'hi',
        tokenizer = tokenizer,
        seq_len = model.config.max_length,
        lang_token_map = LANG_TOKEN_MAPPING)
    input_ids1 = input_ids1.unsqueeze(0).cuda()

    # convert the encoded input text to string and append it to the list
    truncated_input_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids1[0]))
    truncated_inputs.append(truncated_input_text)
    
    # add the corresponding Hindi sentence to the hindi_sentences list
    hindi_sentence = example['translation']['hi']
    input_ids2 = encode_input_str(
        text = hindi_sentence,
        target_lang = 'hi',
        tokenizer = tokenizer,
        seq_len = model.config.max_length,
        lang_token_map = LANG_TOKEN_MAPPING)
    input_ids2 = input_ids2.unsqueeze(0).cuda()
    truncated_hi_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids2[0]))
    hindi_sentences.append(truncated_hi_text)
    
    # print('Truncated input text:', truncated_input_text)


In [ ]:
print((truncated_inputs[0]))

<hi> It has been confirmed that eight thoroughbred race horses at Randwick Racecourse</s>


In [ ]:
output_tokens = model.generate(input_ids1, num_beams=10, num_return_sequences=3)
# print(output_tokens)
for token_set in output_tokens:
  print(tokenizer.decode(token_set, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"ब्रिटिश स्कॉटिश पार्टी का हिस्सा था, लेकिन 
"ब्रिटिश स्कॉटिश एक समूह थे, लेकिन उस
"ब्रिटिश स्कॉटिश एक समूह थे, लेकिन वह


In [ ]:
print((hindi_sentences[0]))

<hi> यह पुष्टि की गई है कि सिडनी में रैंडविक रे</s>


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# initialize an empty list to store the cosine similarity scores
cosine_sim_scores = []

# iterate over the truncated inputs and their corresponding hindi sentences
for i, truncated_sent in enumerate(truncated_inputs):
    # get the corresponding hindi sentence
    hindi_sent = hindi_sentences[i]
    # print(i, hindi_sent)
    # encode the truncated input and hindi sentence
    encoded_truncated_sent = tokenizer(truncated_sent, return_tensors='pt', padding=True, truncation=True)['input_ids']
    encoded_hindi_sent = tokenizer(hindi_sent, return_tensors='pt', padding=True, truncation=True)['input_ids']
    
    # calculate the cosine similarity between the two encoded sentences
    cosine_sim_score = cosine_similarity(encoded_truncated_sent.detach().numpy(), encoded_hindi_sent.detach().numpy())
    
    # append the cosine similarity score to the list
    cosine_sim_scores.append(cosine_sim_score[0][0])
    
    # remove the corresponding hindi sentence if the truncated input contains '###'
    if '###' in truncated_sent:
        hindi_sentences.pop(i)
    
# print the cosine similarity scores
print(cosine_sim_scores)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[0.8105107855293027, 0.8302232661895981, 0.8301411060390835, 0.7407725832692595, 0.8083438585656124, 0.735746583538483, 0.860207678750316, 0.8811497791608564, 0.7900567749876228, 0.7937659793808376, 0.8127204497715261, 0.9525674133634815, 0.7347825235238655, 0.552069180926177, 0.6494796392945662, 0.6931372222812974, 0.7537584987404331, 0.6992856898498911, 0.8675264765107181, 0.5811299232977755, 0.7648991260040459, 0.920694316993307, 0.8616029440886102, 0.6495281377576672, 0.6619763567696112, 0.7175449188110417, 0.8866758574657977, 0.8087935662492567, 0.5944912390608921, 0.50131825177068, 0.9098343833274023, 0.6312500010510171, 0.7004777436080729, 0.9160574501053785, 0.6312197946165382, 0.7039529607993796, 0.6272060565816068, 0.5652399525304524, 0.684509709853392, 0.8889210720816622, 0.8480100307558595, 0.8313782843342016, 0.7131822711593143, 0.8256607562910451, 0.9413941549738108, 0.7702640817157828, 0.701033475579997, 0.9245365781491383, 0.6705615518955059, 0.82138876082593, 0.7805954

In [ ]:
# calculate the average
average = sum(cosine_sim_scores) / len(cosine_sim_scores)

# calculate the median
sorted_numbers = sorted(cosine_sim_scores)
if len(cosine_sim_scores) % 2 == 0:
    median = (sorted_numbers[len(cosine_sim_scores)//2-1] + sorted_numbers[len(cosine_sim_scores)//2]) / 2
else:
    median = sorted_numbers[len(cosine_sim_scores)//2]

# calculate the mode
counts = {}
for num in cosine_sim_scores:
    counts[num] = counts.get(num, 0) + 1
mode = max(counts, key=counts.get)

# print the results
print("Average:", average)
print("Median:", median)
print("Mode:", mode)


Average: 0.7820819241684314
Median: 0.7934837332036824
Mode: 0.8105107855293027


# Testing using sts dataset for Mandarin

In [8]:
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr     

In [12]:
test_en = load_dataset("stsb_multi_mt", name="en", split="test")
# test_zh = load_dataset("stsb_multi_mt", name="en", split="test")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


In [13]:
testEN_sen1 = []
testEN_sen2 = []
test_score = []

for length in range (len(test_en)):
  testEN_sen1.append(test_en[length]["sentence1"].lower())
  testEN_sen2.append(test_en[length]["sentence2"].lower())
  test_score.append(test_en[length]["similarity_score"])

print(len(testEN_sen1), len(testEN_sen2))

1379 1379


In [ ]:
for i in enumerate(testEN_sen1):


In [ ]:
# initialize an empty list to store truncated input texts
hindisen1 = []
truncate1 = []
none_indices = []
hindisen2 = []
truncate2 = []

# iterate over each sentence in the test dataset
i = 0
for a in enumerate(testEN_sen1):


    #SENTENCE 1:
    sent1 = testEN_sen1[i]
    
    if sent1 is None:
        none_indices.append(i)  # add the index to the none_indices list
        continue  # skip over this sentence and move on to the next
    
    # print('Raw input text:', test_sentence)

    # encode the input text and convert it to tensor
    input_ids1 = encode_input_str(
        text = sent1,
        target_lang = 'hi',
        tokenizer = tokenizer,
        seq_len = model.config.max_length,
        lang_token_map = LANG_TOKEN_MAPPING)
    input_ids1 = input_ids1.unsqueeze(0).cuda()

    # convert the encoded input text to string and append it to the list
    truncated_input_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids1[0]))
    truncate1.append(truncated_input_text)

    #translated sent 
    output_tokens = model.generate(input_ids1, num_beams=10, num_return_sequences=3)
    trans1 = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    hindisen1.append(trans1)

    #SENTENCE 2:
    sent2 = testEN_sen2[i]

    input_ids2 = encode_input_str(
        text = sent2,
        target_lang = 'hi',
        tokenizer = tokenizer,
        seq_len = model.config.max_length,
        lang_token_map = LANG_TOKEN_MAPPING)
    input_ids2 = input_ids2.unsqueeze(0).cuda()

    truncated_hi_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids2[0]))
    truncate2.append(truncated_hi_text)

    output_tokens2 = model.generate(input_ids2, num_beams=10, num_return_sequences=3)
    trans2 = tokenizer.decode(output_tokens[1], skip_special_tokens=True)
    hindisen2.append(trans2)

    i = i+1
    
    # print('Truncated input text:', truncated_input_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(len(hindisen1), len(hindisen2))

1379 1379


In [ ]:
print(hindisen1[0])
print(hindisen2[0])

एक लड़के अपने बालों को पसन्द करने का प्रयास कर
एक लड़की अपने बालों को पसंद करने में मदद


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# with open('/content/drive/MyDrive/hindi_Sents.txt', 'w', encoding='utf-8') as f:
#     for sentence in hindisen1:
#         f.write(sentence + '\n')
#     for sentence in hindisen2:
#         f.write(sentence + '\n')


In [6]:
# Open the file in read mode
with open('/content/drive/MyDrive/hindi_Sents.txt', 'r', encoding='utf-8') as f:
    
    # Read the contents of the file
    contents = f.read()
    
    # Split the contents based on the newline character to get the two lists of Hindi sentences
    sentences = contents.strip().splitlines()
    n = len(sentences)
    hindisen1 = sentences[:n//2]
    hindisen2 = sentences[n//2:]
    
# Print the two lists of Hindi sentences
print(len(hindisen1))
print(len(hindisen2))


1379
1379


In [7]:
print(hindisen1[0])
print(hindisen2[0])

एक लड़के अपने बालों को पसन्द करने का प्रयास कर
एक लड़की अपने बालों को पसंद करने में मदद


In [1]:
import gensim
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import FastText


In [2]:
%cd /content/drive/MyDrive


/content/drive/MyDrive


In [4]:
# Load the pre-trained FastText embeddings for Hindi sentences
from gensim.models import KeyedVectors

embeddings = KeyedVectors.load_word2vec_format('cc.hi.300.vec')


In [5]:
def sentence_embedding(sentence, embeddings):
    words = sentence.split()
    vectors = [embeddings[word] for word in words if word in embeddings]
    if not vectors:
        return None
    else:
        return np.mean(vectors, axis=0)


In [14]:
# Compute the sentence embeddings for each list of sentences
sentence_embeddings1 = [sentence_embedding(sentence, embeddings) for sentence in hindisen1]
sentence_embeddings2 = [sentence_embedding(sentence, embeddings) for sentence in hindisen2]

# Compute the similarity scores between the two lists of sentences
similarity_scores = []
for embedding1 in sentence_embeddings1:
    for embedding2 in sentence_embeddings2:
        if embedding1 is None or embedding2 is None:
            similarity_scores.append(0)
        else:
            similarity_scores.append(cosine_similarity([embedding1], [embedding2])[0][0])

# # Compute the average similarity score
# average_similarity = sum(similarity_scores) / len(similarity_scores)
# print(f'The average similarity between the two lists of sentences is {average_similarity:.2f}')


The average similarity between the two lists of sentences is 0.62


In [34]:
similarity_scores = []

for i, embedding1 in enumerate(sentence_embeddings1):
    embedding2 = sentence_embeddings2[i]
    if embedding1 is None or embedding2 is None:
        similarity_scores.append(0)
    else:
        similarity_scores.append(cosine_similarity([embedding1], [embedding2])[0][0])

average_similarity = sum(similarity_scores) / len(similarity_scores)
print(f'The average similarity between the two lists of sentences is {average_similarity:.2f}')


The average similarity between the two lists of sentences is 0.91


In [28]:
print(similarity_scores[23])

0.9147062


In [30]:
len(similarity_scores)

# len(test_score)

1379

In [35]:
# Normalize sim_scores using Min-Max scaling
# scaler = MinMaxScaler(feature_range=(0, 1))
# sim_scores_norm = scaler.fit_transform(np.array(similarity_scores).reshape(-1, 1)).flatten()


# cos_sim and similarity_score are the two lists of similarity scores
# of the same length
assert len(test_score) == len(similarity_scores)

# Calculate the Pearson correlation coefficient and its p-value
corr, p_value = pearsonr(test_score, similarity_scores)

print(f"Pearson correlation coefficient: {corr:.3f}")
print(f"P-value: {p_value:.3f}")


Pearson correlation coefficient: -0.024
P-value: 0.366
